# Intro

This assumes you have gone through the intro/mpi4py intro, and that you are comfortable generating ExaGO output.

Now we will go through specifics to the vizualisation frontend, how it is installed, and a typical workflow.

# Open Issues:

- npm dependencies were pinned and have security issues
- requirements.txt has pinned versions and we cannot upgrade / reproduce
- OpenAI keys and other missing steps...

# Generating file to visualize

I don't know how to do gic file option, so we are going to use OPFLOW C++ CLI to do this...

In [8]:
!opflow --version

ExaGO 1.6.0 built on Nov  9 2023


TODO: Going to let Shri fill the details in here, as likely we don't want people churning large data sets in a small demo, and we can include pre-generated data I would hope... include what gic is 

In [ ]:
!./opflow -netfile <netfile> -save_output -opflow_output_format JSON -gicfile <gicfilename>  

# Visualization

Launch the visualization webpage through shell commands or with the following shell command:

In [1]:
!cd /home/app/viz && . /usr/local/share/nvm/nvm.sh && nvm use 16.13.0 && node --version && npm start

Now using node v16.13.0 (npm v8.1.0)
v16.13.0

> geojson@0.0.0 start
> webpack-dev-server --progress --hot --open

10% building 1/1 modules 0 activeℹ ｢wds｣: Project is running at http://localhost:8080/
ℹ ｢wds｣: webpack output is served from /
ℹ ｢wds｣: Content not from webpack is served from /home/app/viz
ℹ ｢wdm｣: Hash: d8a5654bafde0cba20282m39md.js
Version: webpack 4.47.0
Time: 90373ms
Built at: 11/20/2023 3:38:32 PM
     Asset      Size  Chunks                   Chunk Names
    app.js  45.6 MiB     app  [emitted]        app
app.js.map  20.7 MiB     app  [emitted] [dev]  app
Entrypoint app = app.js app.js.map
[0] multi (webpack)-dev-server/client?http://localhost:8080 (webpack)/hot/dev-server.js ./app.js 52 bytes {app} [built]
[./app.js] 44.1 KiB {app} [built]
[./module_casedata.js] 354 bytes {app} [built]
[./node_modules/@deck.gl/core/dist/esm/index.js] 3.49 KiB {app} [built]
[./node_modules/@deck.gl/extensions/dist/esm/index.js] 795 bytes {app} [built]
[./node_modules/@deck.gl/layers

# ChatGrid

TODO:
- Get pSQL DB working
- Get ChatGPT / external LLM configuration working


# Installation

This is done through the `postCreateCommand` that is configured in `.devcontainer/devcontainer.json` in theroy. Still debugging, so I did some stuff through Jupyter magic just to iterate quickly.

In [2]:
!cat ../../.devcontainer/devcontainer.json

{
	"build": {
		"dockerfile": "Dockerfile",
		"context": ".."
	},
	"customizations": {
		"vscode": {
			"extensions": [
				"ms-azuretools.vscode-docker",
				"github.vscode-github-actions",
				"ms-toolsai.jupyter",
				"GitHub.copilot",
				"ms-vsliveshare.vsliveshare"
			]
		}
	},
	"workspaceFolder": "/home/app/",
  	"postCreateCommand": "cd /home/app/viz && export CXX=$(which gcc) && npm install",
    "workspaceMount": "source=${localWorkspaceFolder},target=/home/app/,type=bind,consistency=cached"
}


In [3]:
!npm --version

9.2.0


In [5]:
!node --version

v18.13.0


In [7]:
!cat ../../.devcontainer/Dockerfile

# Build stage with Spack pre-installed and ready to be used
FROM spack/ubuntu-jammy as builder

# Install OS packages needed to build the software
RUN apt-get -yqq update && apt-get -yqq upgrade \
 && apt-get -yqq install autoconf \
 && rm -rf /var/lib/apt/lists/*

# What we want to install and how we want to install it
# is specified in a manifest file (spack.yaml)
RUN mkdir /opt/spack-environment \
&&  (echo spack: \
&&   echo '  specs:' \
&&   echo '  - exago@1.6.0+ipopt+python+mpi' \
&&   echo '  - petsc~fortran~hdf5~hypre+metis' \
&&   echo '  - ipopt@3.12.10+coinhsl~mumps+metis' \
&&   echo '  - coinhsl@2019.05.21+blas' \
&&   echo '  - py-jupyterlab' \
&&   echo '  - py-ipyparallel' \
&&   echo '  concretizer:' \
&&   echo '    unify: true' \
&&   echo '    reuse: true' \
&&   echo '  mirrors:' \
&&   echo '    spack: https://binaries.spack.io/develop' \
&&   echo '  packages:' \
&&   echo '    all:' \
&&   echo '      providers:' \
&&   echo '        mpi:' \
&&   echo '        